In [126]:
%config IPCompleter.greedy = True # for code auto-completion

In [127]:
from geopy.geocoders import Nominatim
import folium
import pandas as pd
import time
from tqdm import tqdm

In [128]:
geolocator = Nominatim(timeout=10, user_agent = "onos.pkp") # ! an absolute maximum of 1 request per second

In [129]:
# example
location = geolocator.geocode('Wrocław Główny')
display(location)
print(location.latitude)
print(location.longitude)
display(location.raw)

Location(Wrocław Główny, tunel boczny, Glinianki, Osiedle Huby, Wrocław, województwo dolnośląskie, 50-000, Polska, (51.097916, 17.0379512, 0.0))

51.097916
17.0379512


{'place_id': 475548,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 155479759,
 'boundingbox': ['51.092916', '51.102916', '17.0329512', '17.0429512'],
 'lat': '51.097916',
 'lon': '17.0379512',
 'display_name': 'Wrocław Główny, tunel boczny, Glinianki, Osiedle Huby, Wrocław, województwo dolnośląskie, 50-000, Polska',
 'class': 'railway',
 'type': 'station',
 'importance': 0.21100000000000002,
 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/transport_train_station2.p.20.png'}

In [130]:
ic_nodes = pd.read_csv('ic.nodes')
ic_nodes

,id,name
0,0,ALEKSANDRÓW KUJAWSKI
1,1,ANDRYCHÓW
2,2,AUGUSTÓW
3,3,BABIAK
4,4,BABIMOST
...,...,...
454,454,ŻMIGRÓD
455,455,ŻORY
456,456,ŻYCHLIN
457,457,ŻYRARDÓW


In [131]:
get_coordinates_from_api = True # change to True, if you want coordinates update

if get_coordinates_from_api:
    
    data_with_coord = pd.DataFrame(columns=['id','name','latitude','longitude'])

    time_per_request = 1.2

    data = ic_nodes
    # data = ic_nodes.head(4) # for testing
#     data = ic_nodes[150:170]
    
    empty = []
    class Location(object):
        latitude = 0
        longitude = 0
    
    with tqdm(total=len(data.index)) as pbar:
        for index, row in data.iterrows():

            time.sleep(time_per_request)
            location = geolocator.geocode(row['name']+', Polska')  # ! an absolute maximum of 1 request per second
            
            if location is None:
                empty.append(row['name'])
                location = Location()
                
            data_with_coord = data_with_coord.append(
                {'id': row['id'],
                 'name': row['name'],
                 'latitude': location.latitude,
                 'longitude': location.longitude
                }
                , ignore_index=True)
            pbar.update(1)
    
    display(empty) # handle manualy
    # None's: 'MAŁASZEWICZE PRZYSTANEK', 'WARSZAWA ZACHODNIA PERON 8'
    
    data_with_coord.to_csv('ic_coord.nodes', index=False)
    display(data_with_coord)

100%|████████████████████████████████████████████████████████████████████████████████| 459/459 [13:09<00:00,  1.72s/it]


['MAŁASZEWICZE PRZYSTANEK', 'WARSZAWA ZACHODNIA PERON 8']

,id,name,latitude,longitude
0,0,ALEKSANDRÓW KUJAWSKI,52.876537,18.693574
1,1,ANDRYCHÓW,49.855579,19.343676
2,2,AUGUSTÓW,53.843733,22.980236
3,3,BABIAK,52.344523,18.666707
4,4,BABIMOST,52.165312,15.828518
...,...,...,...,...
454,454,ŻMIGRÓD,51.471739,16.908272
455,455,ŻORY,50.038890,18.680612
456,456,ŻYCHLIN,52.244009,19.626093
457,457,ŻYRARDÓW,52.054331,20.443501


In [133]:
ic_nodes_coord = pd.read_csv('ic_coord.nodes')

manual_fix = [('MAŁASZEWICZE PRZYSTANEK', 52.027912, 23.5297023), ('WARSZAWA ZACHODNIA PERON 8', 52.221139, 20.9590913)]

# weird coordinates 

for name, lat, lon in manual_fix:
    ic_nodes_coord.loc[ic_nodes_coord['name'] == name, 'latitude'] = lat
    ic_nodes_coord.loc[ic_nodes_coord['name'] == name, 'longitude'] = lon

ic_nodes_coord.to_csv('ic_coord.nodes', index=False)

display(ic_nodes_coord[165:170])
display(ic_nodes_coord)

,id,name,latitude,longitude
165,165,MALBORK,54.035936,19.034882
166,166,MARCISZÓW,50.852133,16.008824
167,167,MAŁASZEWICZE PRZYSTANEK,52.027912,23.529702
168,168,MAŁKINIA,52.694046,22.033341
169,169,MAŁOMICE,51.555000,15.447291


,id,name,latitude,longitude
0,0,ALEKSANDRÓW KUJAWSKI,52.876537,18.693574
1,1,ANDRYCHÓW,49.855579,19.343676
2,2,AUGUSTÓW,53.843733,22.980236
3,3,BABIAK,52.344523,18.666707
4,4,BABIMOST,52.165312,15.828517
...,...,...,...,...
454,454,ŻMIGRÓD,51.471739,16.908272
455,455,ŻORY,50.038890,18.680612
456,456,ŻYCHLIN,52.244009,19.626093
457,457,ŻYRARDÓW,52.054331,20.443501
